In [1]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
%cd gdrive/MyDrive/Anime_Recommender/Greta_CE

/content/gdrive/MyDrive/Anime_Recommender/Greta_CE


In [ ]:
!pip install sentence_transformers

In [4]:
from sentence_transformers import SentenceTransformer, SentencesDataset, util
from sentence_transformers import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import cross_encoder_train 
import bi_encoder_train 
import recommender
import os
import csv
import pickle
import time
import sys
import pandas as pd
import torch
from torch.utils.data import DataLoader
import math
import gzip
from zipfile import ZipFile
import random
from sentence_transformers import SentencesDataset, losses
from sentence_transformers.evaluation import BinaryClassificationEvaluator
device =torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#Movie lens data - TODO: Get code to work without truncation. Truncation set to 1000 for now. Additionally, all dataset below except ml-20m//genome-tags.csv are preprocessed to match kaggle data set entries
f1='data/movie_lens_merged_out.csv'
f2='data/ratings_out.csv'
f3='data/genome_score_out.csv'
f4='ml-20m//genome-tags.csv'
f5='data/tags_out.csv'
data=recommender.movie_lens_data(f1,f2,f3,f4,f5, truncate=False) 

/content/gdrive/MyDrive/Anime_Recommender/Greta_CE/recommender.py:64: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  generes=generes.str.replace('|',' ')  #removing the "|" seperator


In [6]:
data.bert_tokenize()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [52]:
#This is just for testing. Can be deleted.

user_keywords = "A nobal man discovers that his relative killed his dad and seeks revenge"


recommendations = data.get_recommendations_by_keyword_bert_tfidf(user_keywords, num_recommendations=5, bert_weight=.5, tfidf_weight=.5)
print(recommendations[:5])

['Orwell Rolls in His Grave', 'Boy Meets Boy', 'From Dad to Son', 'Every Girl Should Be Married', 'The Encounter']


In [8]:
# Configure and train Cross-Encoder using Quora Sentence Pairs dataset. Use the filtered 500 synopses as inout to CrossEncoder. A CrossEncoder gets both inputs (input request, synopsis) and outputs similarity score
cross_encoder_model = cross_encoder_train.CE_Model()
cross_encoder_model.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nreimers/MiniLM-L6-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# Configure BiEncoder (SentenceTransformer) that produces embeddings for input request and uses cosine similarity to filter top 500 similar synposes.
bi_encoder_model = bi_encoder_train.BE_Model()
num_candidates = 500


In [10]:
# Dataset we use for BE training. Old - not in use, but current BE trained on this data (with labesl from CE)
# dataset_path= "./data/movies_large.csv"

#Data used for recommender (smaller dataset)
dataset_path = "./data/movies_kaggle_merged.csv"

In [11]:
#Check if dataframe exists
df_cache_path = "df_small.pkl"
if os.path.exists(df_cache_path):
    print("Load pre-computed df from disc")
    df = pd.read_pickle("./df_small.pkl")  
else:
    df = pd.read_csv(dataset_path)
    df.dropna(subset=['overview'], inplace=True)
    df.drop_duplicates(subset='overview', keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)

    print("Store df file on disc")
    df.to_pickle("./df_small.pkl")  

Load pre-computed df from disc


In [12]:
#Prepare CE output to train Bi-Encoder Model
train_num_samples = 10000 #Can adjust this number
qid = [] #query id
train_sentence_pairs = [] 
df_train = df[:train_num_samples]
# print(df_train)
for i in range(train_num_samples):
    rand_num = random.randint(0,train_num_samples-1)
    sentence_1=df_train.iloc[rand_num, 4]
    rand_num = random.randint(0,train_num_samples-1)
    sentence_2=df_train.iloc[rand_num, 4]
    train_sentence_pairs.append([sentence_1, sentence_2])

train_ce_scores = cross_encoder_model.model.predict(train_sentence_pairs)
binary_train_ce_scores=[1 if score>=.1 else 0 for score in train_ce_scores] #Can tune score threshold




In [13]:
#Train Bi-Encoder using Cross-Encoder output and Quora Sentence Pairs dataset
bi_encoder_model.train(binary_train_ce_scores, train_sentence_pairs)

In [14]:
# File for caching  embeddings
embedding_cache_path = 'movies-summary-embeddings-size-small_MNRL.pkl'
embedding_cache_path

'movies-summary-embeddings-size-small_MNRL.pkl'

In [15]:
#Check if embedding cache path exists

if os.path.exists(embedding_cache_path):
    with open(embedding_cache_path, "rb") as fIn:
        cached_data = pickle.load(fIn)
        corpus_sentences = cached_data['corpus_sentences']
        corpus_embeddings = cached_data['corpus_embeddings']
else:
    corpus_sentences = df['overview'].tolist()
    corpus_embeddings = bi_encoder_model.model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True)   
    with open(embedding_cache_path, "wb") as fOut:
        pickle.dump({'corpus_sentences': corpus_sentences, 'corpus_embeddings': corpus_embeddings}, fOut)


    

In [16]:
print(len(df))

14104


In [17]:
print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

Corpus loaded with 14104 sentences / embeddings


In [18]:
print(corpus_embeddings[0:10])
print(corpus_sentences[:10])
print( df['overview'][:10])
print( df['title'][:10])
corpus_titles = set(df['title'].tolist())

tensor([[-0.0276,  0.0158,  0.0286,  ...,  0.0224,  0.0731,  0.0351],
        [-0.0549,  0.0975,  0.0298,  ..., -0.0168, -0.0410, -0.0140],
        [-0.0531,  0.0506,  0.0361,  ..., -0.0174,  0.0817,  0.0139],
        ...,
        [-0.0813,  0.1292,  0.0183,  ..., -0.0828,  0.0454,  0.0180],
        [-0.0457,  0.0412, -0.0659,  ..., -0.0039,  0.0527, -0.0129],
        [-0.0795,  0.0653, -0.0038,  ...,  0.0109, -0.0880, -0.0857]],
       device='cuda:0')
["101-year-old Rose DeWitt Bukater tells the story of her life aboard the Titanic 84 years later. A young Rose boards the ship with her mother and fiancé. Meanwhile Jack Dawson and Fabrizio De Rossi win third-class tickets aboard the ship. Rose tells the whole story from Titanic's departure through to its death—on its first and last voyage—on April 15 1912.", "When the kingdom's most wanted-and most charming-bandit Flynn Rider hides out in a mysterious tower he's taken hostage by Rapunzel a beautiful and feisty tower-bound teen with 70 

In [33]:
filtered_corpus_embedding=[]
filtered_corpus_sentences=[]

index = df.index[df['title']=="Toy Story"][0]
filtered_corpus_sentences.append(corpus_sentences[index])
filtered_corpus_embedding.append(corpus_embeddings[index])

print(filtered_corpus_sentences)

["Led by Woody Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner the duo eventually learns to put aside their differences."]


In [ ]:
"""
WITHOUT Rob's model
"""

while True:
    inp_request = input("Please enter a request: ")
    print("Input request:", inp_request)

    #First, filter 500 candidates using bi-encoder
    start_time = time.time()
    question_embedding = bi_encoder_model.model.encode(inp_request, convert_to_tensor=True)
    
    #Get top 5 results from bi-encoder
    encoder_res = util.semantic_search(question_embedding, corpus_embeddings, top_k=num_candidates)[0]
        #Check that corpus and queries are on the same device
    # if corpus_embeddings.device != question_embedding.device:
    #     query_embeddings = question_embedding.to(corpus_embeddings.device)
    # cos_scores = cos_sim(question_embedding, corpus_embeddings)[0]

    # top_results = torch.topk(cos_scores, k=num_candidates)



    print("Cosine-Similarity search took {:.3f} seconds".format(time.time()-start_time))
    print("Top 5 hits with cosine-similarity:")
    for res in encoder_res[0:5]:
        # print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))
        print("\t{:.3f}\t{}".format(res['score'], df['title'].iloc[res['corpus_id']]))
        # print("\t",df['title'].iloc[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))

    #Use filtered results form bi-encoder as input to cross-encoder
    start_time = time.time()
    sentence_pairs = [[inp_request, corpus_sentences[res['corpus_id']]] for res in encoder_res] #pair all summaries with input request
    ce_scores = cross_encoder_model.model.predict(sentence_pairs)


    for i in range(len(encoder_res)):
        encoder_res[i]['cross-encoder_score'] = ce_scores[i]

    #Sort CrossEncoder output results by scores
    encoder_res = sorted(encoder_res, key=lambda x: x['cross-encoder_score'], reverse=True)
    print("\nRe-ranking with Cross-Encoder took {:.3f} seconds".format(time.time() - start_time))
    print("Top 5 hits with CrossEncoder:")
    for res in encoder_res[0:5]:
        # print("\t{:.3f}\t{}".format(hit['cross-encoder_score'], corpus_sentences[hit['corpus_id']]))
        print("\t{:.3f}\t{}".format(res['cross-encoder_score'], df['title'].iloc[res['corpus_id']]))
    print("\n\n========\n")

In [47]:
from numpy import rec
"""
WITH Rob's model
"""
num_candidates = 500
while True:
    inp_request = input("Please enter a request: ")
    print(f"Input request: {inp_request}")

    #TODO - Add Rob's code here. Not working with current dataset
    start_time = time.time()
    recommendations = data.get_recommendations_by_keyword_bert_tfidf(inp_request, num_recommendations=5000, bert_weight=.8, tfidf_weight=0.2)
    # print(recommendations[:5])    
    # print(len(recommendations))
    print(f"First Cosine-Similarity search took {(time.time()-start_time):.3f} seconds")
    for movie in recommendations[:5]:
        print(f"\t{movie}")
    filtered_corpus_embedding=[]
    filtered_corpus_sentences=[]
    filtered_titles=[]
    titles_seen = set()
    for title in recommendations:
        if title in corpus_titles and title not in titles_seen:
            index = df.index[df['title']==title]
            titles_seen.add(title)
            for idx in index:
                filtered_titles.append(title)
                filtered_corpus_sentences.append(corpus_sentences[idx])
                filtered_corpus_embedding.append(corpus_embeddings[idx])
            # print(title)
            # print(corpus_sentences[index])
    print(f"This is titles length: {len(filtered_titles)}")        
    #First, filter 500 candidates using bi-encoder
    start_time = time.time()
    input_embedding = bi_encoder_model.model.encode(inp_request, convert_to_tensor=True)
    
    #Get top 5 results from bi-encoder
    # encoder_res = util.semantic_search(question_embedding, corpus_embeddings, top_k=num_candidates)[0]
    encoder_res = util.semantic_search(input_embedding, filtered_corpus_embedding, top_k=num_candidates)[0]
    print(f"Second Cosine-Similarity search took {(time.time()-start_time):.3f} seconds")
    print("Top 5 hits with cosine-similarity:")
    for res in encoder_res[0:5]:
        # print(f"\t{(res['score']):.3f}\t{df['title'].iloc[res['corpus_id']]}")
        print(f"\t{(res['score']):.3f}\t{filtered_titles[res['corpus_id']]}")
        # print(f"This is sentence: {filtered_corpus_sentences[res['corpus_id']]}")
    #Use filtered results form bi-encoder as input to cross-encoder
    start_time = time.time()
    sentence_pairs = [[inp_request, filtered_corpus_sentences[res['corpus_id']]] for res in encoder_res] #pair all summaries with input request
    ce_scores = cross_encoder_model.model.predict(sentence_pairs)


    for i in range(len(encoder_res)):
        encoder_res[i]['cross-encoder_score'] = ce_scores[i]

    #Sort CrossEncoder output results by scores
    encoder_res = sorted(encoder_res, key=lambda x: x['cross-encoder_score'], reverse=True)
    print(f"\nRe-ranking with Cross-Encoder took {(time.time()-start_time):.3f} seconds")
    print("Top 5 hits with CrossEncoder:")
    for res in encoder_res[0:5]:
        # print(f"\t{(res['cross-encoder_score']):.3f}\t{df['title'].iloc[res['corpus_id']]}")
        print(f"\t{(res['cross-encoder_score']):.3f}\t{filtered_titles[res['corpus_id']]}")
    print("\n\n========\n")

Please enter a request: A boy's father is killed by his uncle and he seeks revenge.
Input request: A boy's father is killed by his uncle and he seeks revenge.
First Cosine-Similarity search took 0.341 seconds
	Toy Story
	Father of the Bride Part II
	Heat
	Sabrina
	Tom and Huck
This is titles length: 5091
Second Cosine-Similarity search took 0.019 seconds
Top 5 hits with cosine-similarity:
	0.627	Hamlet
	0.603	My Cousin Rachel
	0.602	Pursued
	0.582	Underworld U.S.A.
	0.557	Hesher

Re-ranking with Cross-Encoder took 0.793 seconds
Top 5 hits with CrossEncoder:
	0.041	Road to Perdition
	0.036	Hamlet
	0.011	Tiger Eyes
	0.009	Submarino
	0.005	Out Cold



Please enter a request: A nobal man discovers that his relative killed his dad and seeks revenge
Input request: A nobal man discovers that his relative killed his dad and seeks revenge
First Cosine-Similarity search took 0.341 seconds
	Waiting to Exhale
	Father of the Bride Part II
	Sabrina
	Tom and Huck
	Balto
This is titles length: 5124
Se

KeyboardInterrupt: ignored